#### CSCI-UA.0473-​001 Introduction to Machine Learning

# Homework 3


### Name: (your name goes here)


### Due: Oct. 28, 2020


## Goal:  The goal of this homework is to practice implementing kernels for support vector machine and to practice using cross-validation.

Please DO NOT change the position of any cell in this assignment. If the notebook is stunned sometimes before the cell that defines SVM class, please restart it.

You will need the following packages below to do the homework.  Please DO NOT import any other packages.

## WARNING!
Some parts below (especially with cross-validation) could take a while to run ~10-15 min.  If it takes much longer than this, then you likely have an error.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from scipy.optimize import minimize
import matplotlib.pyplot as plt

### Loading and spliting data (nothing to do here)
For this assignment, we utilized the iris dataset from sklearn library. Most of the setting is the same as homework 2, except the training/test are split in $70\%/30\%$.
* `X_train` and `y_train` are `features` and `labels` for training, while `X_test` and `y_test` are for testing.

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

X = X[y != 0, :2] # Only use the first two features.
y = y[y != 0]     # Ignore the first class.
y[y==2] = -1      # Change class label to -1 for SVM.

n_sample = len(X) # Total number of data points.

# Split data into training and testing sets.
#np.random.seed(0)
order = np.random.permutation(n_sample)
X = X[order]
y = y[order].astype(np.float)

X_train = X[:int(.7 * n_sample)]
y_train = y[:int(.7 * n_sample)]
X_test = X[int(.7 * n_sample):]
y_test = y[int(.7 * n_sample):]

## 1. Kernel SVM (75 pts total)

### Part (a) (15 pts) Implementing kernel functions
Recall that in homework 2 we only used the inner product $\langle x_1,x_2\rangle$ as the kernel. This time we are going implement three new kernels:

1. `linear`: $k(x_1,x_2)=x_1^T x_2$.  Note that this is what you did in the previous homework and is already done for you.

2. `poly`: $k(x_1,x_2)=\left(x_1^T x_2 + 1\right)^2$

3. `rbf`: $k(x_1,x_2) = \exp\left( -\frac{1}{2}\|x_1 - x_2\|^2 \right)$.

4. `laplace`: $k(x_1,x_2)=\exp\left(-\frac{1}{2}\left\| x_1 - x_2 \right\| \right)$

Complete the function below by implementing the kernel function for each case.

In [ ]:
def kernel_product(x1, x2, kernel = 'linear'):
    """
    Compute the kernelal product k(x1,x2) for different choices of the kernel k.
    
    Input:
        x1: np.ndarray(p,), the first vector
        x2: np.ndarray(p,), the second vector
        kernel: str, a string which is the name of the kernel, must match one of the options below exactly
        linear, poly, rbf, laplace
    
    Return:
        k: float, the value of the kernel k(x1, x2)
    """
    
    ##TODO-start##
    if kernel == 'linear':
        k = np.dot(x1, x2)
        return k
    elif kernel == 'poly':
        k = ?
        return k
    elif kernel == 'rbf':
        k = ?
        return k
    elif kernel == 'laplace':
        k = ?
        return k
    ## TODO-end##
    else:
        print("Invalid kernel: {:s}".format(kernel))

The next cell is just a helper method similar to the previous homework.  You do not need to implement anything in it.

In [ ]:
def kernel_product_matrix(X, kernel = 'linear'):
    """
    Compute the inner product matrix of two vectors.
    
    Input:
        X: np.ndarray(n,p), data matrix with n data points (each row) and p features
        kernel: str, a string which is the name of the kernel, must match one of the options below exactly
        linear, poly, rbf, laplace
    
    Return:
        K: np.ndarray(n,n), each entry is the kernel product of the corresponding pair of vectors
    """
    n = len(X)
    K = np.zeros((n, n))  
    for i in range(n):
        for j in range(i, n):
            K[i, j] = kernel_product(X[i], X[j], kernel)
            K[j, i] = K[i, j] # Matrix is symmetric so we can cut computation in half.          
    return K

### Part (b) (25 pts) Finishing the kernel SVM implementation

Now you will extend your SVM implementation from the previous homework by including kernels.  Finish the SVM class below by filling in the missing lines of code.  You need to do 3 things, all of which are very similar to the previous homework.

1. Implement the objective function for the dual problem for minimization.  Note that we actually maximize the dual objective function, but in order to use the `minimize()` function from Sci-Py you will need to take the negative.

2. Compute the bias.  Use your implementation from the previous homework for guidance and think carefully about what needs to change.

3. Implement the `predict()` function.

In [ ]:
class SVM:
    
    
    def __init__(self, C = 1, kernel = 'linear'):
        """
        Initialize the SVM model.
        
        Input:
            C: float, the regularization constant for SVM.
            kernel: str, a string which is the name of the kernel, must match one of the options below exactly
            linear, poly, rbf, laplace
        
        Return:
            None
        """
        assert C >= 0
        self.C = C
        self.kernel = kernel
        # The following variables are set after fit() is called.
        self.X_train = None
        self.y_train = None
        self.bias  = None
        self.alpha = None
     
    
    def fit(self, X_train, y_train):
        """
        Computes the parameters alpha and bias that determine the maximum-margin decision boundary for SVM.
        bias will be a float, alpha is a np.ndarray(n,) vector of the dual variables.
    
        Input:
            X_train: np.ndarray(n,p), matrix of training data features
            y_train: np.ndarray(n, ), vector of training data labels
        
        Return:
            None
        """
        # Save the training data.
        self.X_train = X_train
        self.y_train = y_train
        # Number of training points and dimension.
        n, p = X_train.shape
        
        # Get the negative objective function to change maximization to minimization.
        ##TODO-start##
        W = ?
        ##TODO-end##
                                                     
        # Initialization and constraints for optimization.
        init_pt = np.zeros(n)
        bnds = tuple([(0, self.C) for i in range(n)])
        cons = ({'type': 'eq', 'fun': lambda x:  np.dot(x, y_train)})
        # Solve the dual problem for SVM.
        res = minimize(W, init_pt, method='SLSQP', bounds=bnds,
               constraints=cons)   
        alpha = res.x
        self.alpha = alpha
                                                     
        # Compute the bias
        ##TODO-start##
        self.bias = ?
        ##TODO-end##
    
                                                     
    def predict(self, X_test):
        """
        Compute the predictions y_pred on the test set using only the support vectors.
    
        Input:
            X_test: np.ndarray(n,p), matrix of the test data
    
        Return:
            y_pred: np.ndarray(n,), vector of the predicted labels, either +1 or -1
        """
        ##TODO-start##
        y_pred = ?
        ##TODO-end##
        return y_pred

The next cell is taken from the previous homework.  There is nothing for you to do here.

In [ ]:
def accuracy(y_pred, y_test):
    """
    Computes the accuracy on the test set given the class predictions.
    
    Input:
        y_pred: np.ndarray(n,), vector of predicted class labels
        y_test: np.ndarray(n,), vector of true class labels
    
    Output:
        float, accuracy of predictions
    """
    return np.mean(y_pred*y_test > 0) 

### Draw the decision boundaries of all kernels (nothing to do here)
The following is an illustration of decision boundaries for SVM with kernels. You do not need to do anything in the next cell.  It is only to check your work.

In [ ]:
kernel_list = ['linear', 'rbf', 'poly', 'laplace']

for kernel in kernel_list:
    model = SVM(C=10, kernel=kernel)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("SVM with {:s} kernel, accuracy = {:0.2f}%".format(kernel, 100*accuracy(y_pred, y_test)))

    plt.figure()
    plt.clf()
    plt.scatter(X[:, 0], X[:, 1], c=y, zorder=10, cmap=plt.cm.Paired,
                edgecolor='k', s=20)

    # Circle out the test data
    plt.scatter(X_test[:, 0], X_test[:, 1], s=80, facecolors='none',
                zorder=10, edgecolor='k')

    plt.axis('tight')
    x_min = X[:, 0].min()
    x_max = X[:, 0].max()
    y_min = X[:, 1].min()
    y_max = X[:, 1].max()

    XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
    XXYY = np.c_[XX.ravel(), YY.ravel()]
    Z = model.predict(XXYY)

    # Put the result into a color plot
    Z = Z.reshape(XX.shape)
    plt.pcolormesh(XX, YY, Z > 0, cmap=plt.cm.Paired)
    plt.title(kernel)

plt.show()

### Part (c) (20 pts) Cross-validation to choose the regularization parameter $C$

Complete the cross-validation function below.  The data has already been randomly arranged for you.  There are 4 things you will need to do for each fold.

1. Split the data.
2. Train the model.
3. Get the predictions.
4. Compute the accuracy.

Once you have finished run the following cell to check your work.

In [ ]:
def cross_validation(model, X_train, y_train, folds = 5):
    """
    Perform k-fold cross-validation on model using the available training data.  You may assume that 
    the number of training data points is divisible by the number of folds.
    
    Input:
        model: SVM object, an instance of the SVM class, must have fit and predict implemented.
        X_train: np.ndarray(n,p), training data features
        y_train: np.ndarray(n,), training data labels
        folds: int, number of cross-validation folds to perform
    
    Output:
        acc: float, the mean accuracy from all cross-validation folds
        acc_results: np.ndarray(folds,), the accuracy results from each cross-validation fold
    """
    n = X_train.shape[0] # Number of available training data points.
    acc_results = np.zeros(folds) # Store the cross-validation results here.
    
    # Randomly permute the data.
    permutation = np.random.permutation(n)
    X_train = X_train[permutation, :]
    y_train = y_train[permutation]
    
    for k in range(folds):
        print("Fold {:d} / {:d} is running.".format(k, folds))
        
        ##TODO-start##

        ##TODO-end##
    
    acc = np.mean(acc_results)
    return (acc, acc_results)

Run the next cell to check your work.  There is nothing you need to implement here.

In [ ]:
C_list = [0.001, 0.01, 0.1, 0.5, 1, 5]

for c in C_list:
    print('Cross-validation for C = {:0.3f}'.format(c))
    model = SVM(C = c, kernel = 'rbf')
    acc, acc_results = cross_validation(model, X_train, y_train)
    print('Mean = {:0.2f}%'.format(100*acc))

### Part (d) (10 pts) Mean and standard deviation of cross-validation scores

Now use 10-fold cross-validation on two models: 1. SVM with RBF kernel and 2. SVM with the linear kernel.  Set the regularization parameter $C = 0.5$ for both models.  Print out the mean accuracy and the standard deviation of the accuracies from each fold using the numpy functions `np.mean()` and `np.std()`.  Format your results as percentages to two decimal places using Python's `format()` function for strings so that it is easy to read.  Make sure the values you are printing are clearly indicated.

In [ ]:
##TODO-start##
print('Cross-validation for SVM with RBF kernel')


print('Cross-validation for SVM with linear kernel')

##TODO-end##

### Part (e) (10 pts) Generalization error

Remember that the reason for why we do cross-validation is because we want an estimate of the generalization error of our model.
$$
    E = \mathbb{E}_{(X,Y)}\left[ \mathbf{1}\{ f(X) \neq Y \} \right]
$$
where the indicator function
$$
    \mathbf{1}\{ f(X) \neq Y \} = \begin{cases} 
    1 & f(X) \neq Y\\
    0 & f(X) = Y
    \end{cases}
$$
and $f(X)$ is our prediction for the test point $X$.  Remember that the expecation is taken over the joint distribution of $(X,Y)$.  Since we only have $n$ training data pairs $\{(x_i, y_i)\}_{i=1}^n$ we approximate the expecation with a sample average
$$
    \hat{E}_n = \frac{1}{n} \sum_{i=1}^n \mathbf{1}\{ f(x_i) \neq y_i \}
$$
Note that $f$ itself is learned (fitted) using the training points.  Suppose that we have $n$ available data points $\{(x_i,y_i)\}_{i=1}^n$ and that we split the data into a training set $D_{\text{train}} = \{(x_j, y_j)\}_{j=1}^{n_{\text{train}}}$ of $n_{\text{train}}$ points as well as a validation set $D_{\text{val}} = \{(x_i, y_i)\}_{i=n_{\text{train}} + 1}^{n_{\text{train}} + n_{\text{val}}}$ of $n_{\text{val}}$ points with $n_{\text{val}} + n_{\text{train}} = n$.  Now answer the following two questions.


1. Is it true that if $n_{\text{val}} \to \infty$ then $\hat{E}_{n_{\text{val}}} \to E$?  Here $\hat{E}_{n_{\text{val}}}$ is computed using only the validation points.  You do not need a formal proof, but provide a clear explanation.  Hint: you may want to remind yourself of the law of large numbers.

2. Give a heuristic explanation as to why $\hat{E}_{n_{\text{val}}}$ is a better estimator of $E$ than $\hat{E}_{n_{\text{train}}}$ (error computed only training points) assuming that $n_{\text{val}} = n_{\text{train}}$.

**Answer goes here**


## 2. Finding a feature map (15 pts total)

Consider the following dataset shown below, which is just two concentric circles centered at $0$ of radius 1 and 2 corresponding to the two different classes.  There is nothing for you to implement in the cell below.  It is only for visualization.

In [ ]:
# Plot the toy dataset which is just two circles of radius 1 and 2.

theta = np.linspace(0, 2*np.pi, 20) # Angles.

# Class 0
x0 = np.cos(theta)
y0 = np.sin(theta)

# Class 1
x1 = 2*np.cos(theta)
y1 = 2*np.sin(theta)

# Plot the data.
plt.figure()
plt.plot(x0, y0, 'b x', label = 'Class 0')
plt.plot(x1, y1, 'r s', label = 'Class 1')
plt.axis('equal')
plt.title('Toy Dataset')
plt.legend()
plt.show();

Clearly we should be able to easily separate this data to get a classifier with $100\%$ accuracy since there is no overlap.  Answer the following questions.

1.  Write down a feature map $\phi : \mathbb{R}^2 \to \mathbb{R}$, such that the transformed data $\phi(x)$ is linearly separable.  

2.  What is the corresponding kernel?

3.  What is the decision boundary on the original space?

**Answer goes here**:



## 3. Verifying the Polynomial Kernel (10 pts total)

Prove that the polynomial kernel $K(u,v) = (u^Tv + 1)^d$ for $d \ge 1$ and $d$ integer is a valid kernel.  This means you need to verify the assumptions of Mercer's theorem that

1. $K$ is symmetric.

2. $K$ is positive semi-definite.

**Answer goes here**
